# Get Params on Training Data

In [32]:
import pandas as pd
import numpy as np
from autogluon.multimodal import MultiModalPredictor
from sklearn.utils.class_weight import compute_class_weight
import datetime
import os
import torch

In [33]:
torch.set_float32_matmul_precision("medium")

In [34]:
# Load train and test datasets
train_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/train_df.csv")
test_df = pd.read_csv("E:/Current_Workdir/palm-fruit-classification/data/clean/test_df.csv")

In [35]:
# Compute class weights
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(train_df["label_text"]), y=train_df["label_text"])
class_weights_list = class_weights.tolist()

In [36]:
# Hyperparameters for each model
hyperparam_1 = {
    "Model": "EfficientNet", "gamma": 2.255197916, "learning rate": 0.000198215, "batch_size": 16, "epoch": 13
}
hyperparam_2 = {
    "Model": "EfficientNet", "gamma": 2.49282, "learning rate": 0.000128404, "batch_size": 128, "epoch": 7
}
hyperparam_3 = {
    "Model": "EfficientNet", "gamma": 2.60701, "learning rate": 0.000942346, "batch_size": 16, "epoch": 8
}
hyperparam_4 = {
    "Model": "ResNet18", "gamma": 2.101006332, "learning rate": 0.000719425, "batch_size": 8, "epoch": 9
}
hyperparam_5 = {
    "Model": "ResNet18", "gamma": 2.45143, "learning rate": 0.000269354, "batch_size": 256, "epoch": 7
}
hyperparam_6 = {
    "Model": "ResNet18", "gamma": 2.10302, "learning rate": 0.000678854, "batch_size": 8, "epoch": 9
}
hyperparam_7 = {
    "Model": "EfficientNetv2", "gamma": 2.105304731, "learning rate": 0.000357808, "batch_size": 16, "epoch": 7
}
hyperparam_8 = {
    "Model": "EfficientNetv2", "gamma": 2.62577, "learning rate": 0.000262012, "batch_size": 64, "epoch": 18
}
hyperparam_9 = {
    "Model": "EfficientNetv2", "gamma": 2.25188, "learning rate": 0.000339079, "batch_size": 128, "epoch": 6
}

In [37]:
# Helper function to create the model path and define hyperparameters
def create_and_train_model(hyperparam, idx):
    model_name = hyperparam['Model']
    gamma = hyperparam['gamma']
    learning_rate = hyperparam['learning rate']
    batch_size = hyperparam['batch_size']
    epochs = hyperparam['epoch']

    # Set model-specific identifier
    if model_name == "EfficientNet":
        model_checkpoint = "efficientnet_b2"
    elif model_name == "ResNet18":
        model_checkpoint = "resnet18"
    elif model_name == "EfficientNetv2":
        model_checkpoint = "tf_efficientnetv2_s.in1k"
    else:
        raise ValueError(f"Unknown model: {model_name}")

    # Set training folder name
    now = datetime.datetime.now()
    timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S")
    model_path = f"../../model/master/train_{model_name}_{idx}_{timestamp_str}"
    os.makedirs(model_path, exist_ok=True)

    # Define hyperparameters
    hyperparameters = {
        "optimization.learning_rate": learning_rate,
        "optimization.max_epochs": int(epochs),
        "env.batch_size": int(batch_size),
        "optimization.loss_function": "focal_loss",
        "optimization.focal_loss.alpha": class_weights_list,
        "optimization.focal_loss.gamma": gamma,
        "optimization.focal_loss.reduction": "sum",
        "model.timm_image.checkpoint_name": model_checkpoint,
        "optimization.optim_type": "adamw",
        "optimization.top_k_average_method": "best",
    }

    # Train model
    predictor = MultiModalPredictor(
        label="label_text",
        path=model_path,
        problem_type="multiclass",
        eval_metric="f1_macro",
    )
    predictor.fit(train_data=train_df, hyperparameters=hyperparameters)

    # Evaluate model on train_df
    train_scores = predictor.evaluate(train_df, metrics=["f1_macro", "f1_micro", "accuracy", "recall_macro", "precision_macro"])
    
    # Print all train scores
    print(f"Model {model_name} (hyperparameters {idx}) train scores:")
    for metric, score in train_scores.items():
        print(f"  {metric}: {score}")

In [38]:
# Train and evaluate each model with its respective hyperparameters
create_and_train_model(hyperparam_1, 1)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       103.15 GB / 127.91 GB (80.6%)
Disk Space Avail:   1396.72 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_1_2024-12-29_17-30-25
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:12<00:12, 10.96it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.37721 (best 0.37721), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_1_2024-12-29_17-30-25\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:25<00:00, 10.68it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.61362 (best 0.61362), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_1_2024-12-29_17-30-25\\epoch=0-step=134.ckpt' as top 3


...

Epoch 12: 100%|██████████| 269/269 [00:27<00:00,  9.92it/s]

`Trainer.fit` stopped: `max_epochs=13` reached.


Epoch 12: 100%|██████████| 269/269 [00:27<00:00,  9.92it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.51it/s]
Model EfficientNet (hyperparameters 1) train scores:
  f1_macro: 0.9993290633412034
  f1_micro: 0.9988674971687429
  accuracy: 0.9988674971687429
  recall_macro: 0.9994979919678716
  precision_macro: 0.9991619526503248


In [39]:
create_and_train_model(hyperparam_2, 2)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       106.09 GB / 127.91 GB (82.9%)
Disk Space Avail:   1396.69 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_2_2024-12-29_17-37-14
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:12<00:12, 10.84it/s]                 

Epoch 0, global step 8: 'val_f1_macro' reached 0.22420 (best 0.22420), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_2_2024-12-29_17-37-14\\epoch=0-step=8.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:26<00:00, 10.16it/s]

Epoch 0, global step 16: 'val_f1_macro' reached 0.43901 (best 0.43901), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_2_2024-12-29_17-37-14\\epoch=0-step=16.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:27<00:00,  9.95it/s]

`Trainer.fit` stopped: `max_epochs=7` reached.


Epoch 6: 100%|██████████| 269/269 [00:27<00:00,  9.94it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.67it/s]
Model EfficientNet (hyperparameters 2) train scores:
  f1_macro: 0.8794564785000265
  f1_micro: 0.8501321253303133
  accuracy: 0.8501321253303133
  recall_macro: 0.9149967358260773
  precision_macro: 0.8522810365755951


In [40]:
create_and_train_model(hyperparam_3, 3)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.95 GB / 127.91 GB (82.8%)
Disk Space Avail:   1396.66 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNet_3_2024-12-29_17-41-15
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args

Epoch 0:  50%|████▉     | 134/269 [00:13<00:13,  9.98it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.48908 (best 0.48908), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_3_2024-12-29_17-41-15\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:27<00:00,  9.79it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.54968 (best 0.54968), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNet_3_2024-12-29_17-41-15\\epoch=0-step=134.ckpt' as top 3


...

Epoch 7: 100%|██████████| 269/269 [00:27<00:00,  9.79it/s]

`Trainer.fit` stopped: `max_epochs=8` reached.


Epoch 7: 100%|██████████| 269/269 [00:27<00:00,  9.79it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:03<00:00, 22.86it/s]
Model EfficientNet (hyperparameters 3) train scores:
  f1_macro: 0.9984310589738629
  f1_micro: 0.9977349943374858
  accuracy: 0.9977349943374858
  recall_macro: 0.9980295330983798
  precision_macro: 0.9988425925925926


In [41]:
create_and_train_model(hyperparam_4, 4)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.97 GB / 127.91 GB (82.8%)
Disk Space Avail:   1396.64 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_4_2024-12-29_17-45-47
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:04<00:04, 29.67it/s]                 

Epoch 0, global step 134: 'val_f1_macro' reached 0.38448 (best 0.38448), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_4_2024-12-29_17-45-47\\epoch=0-step=134.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:09<00:00, 27.09it/s]

Epoch 0, global step 268: 'val_f1_macro' reached 0.62221 (best 0.62221), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_4_2024-12-29_17-45-47\\epoch=0-step=268.ckpt' as top 3


...

Epoch 8: 100%|██████████| 269/269 [00:10<00:00, 25.66it/s]

`Trainer.fit` stopped: `max_epochs=9` reached.


Epoch 8: 100%|██████████| 269/269 [00:10<00:00, 25.65it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 36.89it/s]
Model ResNet18 (hyperparameters 4) train scores:
  f1_macro: 0.9943624300623473
  f1_micro: 0.9913174782936958
  accuracy: 0.9913174782936958
  recall_macro: 0.9947390006424053
  precision_macro: 0.9940005160754275


In [42]:
create_and_train_model(hyperparam_5, 5)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.77 GB / 127.91 GB (82.7%)
Disk Space Avail:   1396.60 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_5_2024-12-29_17-48-15
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:03<00:03, 39.47it/s]                 

Epoch 0, global step 4: 'val_f1_macro' reached 0.13291 (best 0.13291), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_5_2024-12-29_17-48-15\\epoch=0-step=4.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:07<00:00, 34.56it/s]

Epoch 0, global step 8: 'val_f1_macro' reached 0.17638 (best 0.17638), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_5_2024-12-29_17-48-15\\epoch=0-step=8.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:08<00:00, 31.70it/s]

`Trainer.fit` stopped: `max_epochs=7` reached.


Epoch 6: 100%|██████████| 269/269 [00:08<00:00, 31.69it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 39.24it/s]
Model ResNet18 (hyperparameters 5) train scores:
  f1_macro: 0.5503866838143608
  f1_micro: 0.5285013212533032
  accuracy: 0.5285013212533032
  recall_macro: 0.6999065965043245
  precision_macro: 0.5365350870934883


In [43]:
create_and_train_model(hyperparam_6, 6)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.63 GB / 127.91 GB (82.6%)
Disk Space Avail:   1396.55 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_ResNet18_6_2024-12-29_17-50-07
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)

Epoch 0:  50%|████▉     | 134/269 [00:04<00:04, 28.11it/s]                 

Epoch 0, global step 134: 'val_f1_macro' reached 0.38905 (best 0.38905), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_6_2024-12-29_17-50-07\\epoch=0-step=134.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:10<00:00, 26.17it/s]

Epoch 0, global step 268: 'val_f1_macro' reached 0.63428 (best 0.63428), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_ResNet18_6_2024-12-29_17-50-07\\epoch=0-step=268.ckpt' as top 3


...

Epoch 8: 100%|██████████| 269/269 [00:10<00:00, 25.24it/s]

`Trainer.fit` stopped: `max_epochs=9` reached.


Epoch 8: 100%|██████████| 269/269 [00:10<00:00, 25.24it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:02<00:00, 34.73it/s]
Model ResNet18 (hyperparameters 6) train scores:
  f1_macro: 0.9948089610154413
  f1_micro: 0.9924499811249529
  accuracy: 0.9924499811249529
  recall_macro: 0.9962074595118967
  precision_macro: 0.9934341292480008


In [44]:
create_and_train_model(hyperparam_7, 7)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.13 GB / 127.91 GB (82.2%)
Disk Space Avail:   1396.51 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_7_2024-12-29_17-52-39
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:16<00:16,  8.24it/s]                 

Epoch 0, global step 67: 'val_f1_macro' reached 0.65808 (best 0.65808), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_7_2024-12-29_17-52-39\\epoch=0-step=67.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:34<00:00,  7.73it/s]

Epoch 0, global step 134: 'val_f1_macro' reached 0.79181 (best 0.79181), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_7_2024-12-29_17-52-39\\epoch=0-step=134.ckpt' as top 3


...

Epoch 6: 100%|██████████| 269/269 [00:40<00:00,  6.71it/s]

`Trainer.fit` stopped: `max_epochs=7` reached.


Epoch 6: 100%|██████████| 269/269 [00:40<00:00,  6.71it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.51it/s]
Model EfficientNetv2 (hyperparameters 7) train scores:
  f1_macro: 0.9991482527935872
  f1_micro: 0.9984899962249906
  accuracy: 0.9984899962249906
  recall_macro: 0.9992768275900806
  precision_macro: 0.9990204006708766


In [45]:
create_and_train_model(hyperparam_8, 8)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.20 GB / 127.91 GB (82.2%)
Disk Space Avail:   1396.43 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_8_2024-12-29_17-58-15
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:17<00:17,  7.57it/s]                 

Epoch 0, global step 16: 'val_f1_macro' reached 0.31392 (best 0.31392), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_8_2024-12-29_17-58-15\\epoch=0-step=16.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:37<00:00,  7.13it/s]

Epoch 0, global step 33: 'val_f1_macro' reached 0.56787 (best 0.56787), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_8_2024-12-29_17-58-15\\epoch=0-step=33.ckpt' as top 3


...

Epoch 16:  50%|████▉     | 134/269 [00:15<00:15,  8.75it/s]

Epoch 16, global step 560: 'val_f1_macro' was not in top 3


Epoch 16:  50%|████▉     | 134/269 [00:16<00:16,  8.11it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.68it/s]
Model EfficientNetv2 (hyperparameters 8) train scores:
  f1_macro: 0.999638471753412
  f1_micro: 0.9992449981124952
  accuracy: 0.9992449981124952
  recall_macro: 0.999665327978581
  precision_macro: 0.9996124031007753


In [46]:
create_and_train_model(hyperparam_9, 9)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          32
Pytorch Version:    2.4.1
CUDA Version:       12.4
Memory Avail:       105.54 GB / 127.91 GB (82.5%)
Disk Space Avail:   1396.36 GB / 1863.00 GB (75.0%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir e:\Current_Workdir\palm-fruit-classification\autogluon_ver\model\master\train_EfficientNetv2_9_2024-12-29_18-09-34
    ```

Seed set to 0
GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\lightning\pytorch\plugins\precision\amp.py:52: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', ar

Epoch 0:  50%|████▉     | 134/269 [00:15<00:15,  8.75it/s]                 

Epoch 0, global step 8: 'val_f1_macro' reached 0.39521 (best 0.39521), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_9_2024-12-29_18-09-34\\epoch=0-step=8.ckpt' as top 3


Epoch 0: 100%|█████████▉| 268/269 [00:32<00:00,  8.17it/s]

Epoch 0, global step 16: 'val_f1_macro' reached 0.64665 (best 0.64665), saving model to 'E:\\Current_Workdir\\palm-fruit-classification\\autogluon_ver\\model\\master\\train_EfficientNetv2_9_2024-12-29_18-09-34\\epoch=0-step=16.ckpt' as top 3


...

Epoch 5: 100%|██████████| 269/269 [00:38<00:00,  6.93it/s]

`Trainer.fit` stopped: `max_epochs=6` reached.


Epoch 5: 100%|██████████| 269/269 [00:38<00:00,  6.93it/s]


e:\Programs\miniforge3\envs\autogluon_stable_112\Lib\site-packages\autogluon\multimodal\utils\checkpoint.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  avg_state_dict 

Predicting DataLoader 0: 100%|██████████| 83/83 [00:07<00:00, 11.55it/s]
Model EfficientNetv2 (hyperparameters 9) train scores:
  f1_macro: 0.9926517254607311
  f1_micro: 0.9890524726311816
  accuracy: 0.9890524726311816
  recall_macro: 0.9949050280375581
  precision_macro: 0.9904759836230589
